## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_maps_api

# Configure gmaps API key
gmaps.configure(api_key=google_maps_api)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,cherskiy,RU,68.7500,161.3000,273.59,85,20,4.27,few clouds
1,1,kupang,ID,-10.1667,123.5833,299.47,79,60,4.66,broken clouds
2,2,hermanus,ZA,-34.4187,19.2345,289.15,52,100,4.18,overcast clouds
3,3,faanui,PF,-16.4833,-151.7500,298.25,77,25,9.79,light rain
4,4,morondava,MG,-20.2833,44.2833,298.75,77,63,3.93,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 280
What is the maximum temperature you would like for your trip? 290


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,hermanus,ZA,-34.4187,19.2345,289.15,52,100,4.18,overcast clouds
6,6,hofn,IS,64.2539,-15.2082,281.89,77,10,4.57,clear sky
7,7,jamestown,US,42.0970,-79.2353,281.92,51,40,6.17,scattered clouds
9,9,severo-kurilsk,RU,50.6789,156.1250,283.35,75,98,2.42,overcast clouds
10,10,marathon,US,44.9000,-89.7668,285.48,80,0,2.57,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,hermanus,ZA,289.15,overcast clouds,-34.4187,19.2345,
6,hofn,IS,281.89,clear sky,64.2539,-15.2082,
7,jamestown,US,281.92,scattered clouds,42.0970,-79.2353,
9,severo-kurilsk,RU,283.35,overcast clouds,50.6789,156.1250,
10,marathon,US,285.48,clear sky,44.9000,-89.7668,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_maps_api
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass

hotel_df.head()        

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,hermanus,ZA,289.15,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
6,hofn,IS,281.89,clear sky,64.2539,-15.2082,Hótel Höfn
7,jamestown,US,281.92,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
9,severo-kurilsk,RU,283.35,overcast clouds,50.6789,156.1250,Kak Doma
10,marathon,US,285.48,clear sky,44.9000,-89.7668,


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,hermanus,ZA,289.15,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
6,hofn,IS,281.89,clear sky,64.2539,-15.2082,Hótel Höfn
7,jamestown,US,281.92,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
9,severo-kurilsk,RU,283.35,overcast clouds,50.6789,156.1250,Kak Doma
11,jaguarao,BR,288.79,clear sky,-32.5661,-53.3758,Hotel Rios


In [16]:
# 8a. Create the output File (CSV)
output_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} \u00B0F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))